# Applicazione Python "Premi Nobel"

Attività scelta: 3. MongoDB Usecase e applicazione Python <br>
Progetto realizzato da Elena Curti (matricola: 146539 <mark>FORSE DA CAMBIARE?</mark>) con <mark> Python 3.11.0 (installato) oppure Python 3.9.13 (anaconda) ?

Ho usato i seguenti file JSON pubblici: <br>
    <ul> 
      <li> nobelPrizes.json: https://masterdataapi.nobelprize.org/2.1/nobelPrizes?offset=0&limit=664 </li>
      <li> laureates.json: https://masterdataapi.nobelprize.org/2.1/laureates?offset=0&limit=981 </li>
    </ul>
Essi contengono informazioni sui premi Nobel ed i relativi vincitori.

## Documentazione 
Documentazione dei dataset: https://app.swaggerhub.com/apis/NobelMedia/NobelMasterData/2.1

## File con i premi Nobel
Il file <i>JSON_Files/modificati/nobelPrizes2.json</i> contiene l'elenco dei premi Nobel, con la seguente struttura d'esempio:

```
{
    "awardYear": 2021,
    "category": "Economic Sciences",
    "categoryFullName": "The Sveriges Riksbank Prize in Economic Sciences in Memory of Alfred Nobel",
    "dateAwarded": "2021-10-11",
    "topMotivation": "for contributions ...",
    "prizeAmount": 10000000,
    "laureates": [
        {
            "id": "1007",
            "knownName": "David Card", // Solo per le persone
            "orgName": "Institute of International Law", // Solo per le organizzazioni
            "portion": "1/2",
            "sortOrder": "1",
            "motivation": "for his empirical contributions to labour economics"
        },
        { ... }
    ]
}
```

### Modifiche effettuate
Rispetto al file JSON originale (contenuto in <i>JSON_Files/originali/nobelPrizes.json</i>): 
<ul>
    <li> Ho rimosso i sottoalberi "/link" e "/meta" perchè contenenti informazioni non inerenti ai premi Nobel. <br></li>
    <li> 
        Ho rimosso la traduzione di alcuni campi. La struttura originale di tali campi era la seguente: 

```
"nome": {
    "en": "termine_inglese",
    "no": "termine_svedese",
    "se": "termine_norvegese"
}

```

<ul> 
<li style="list-style-type: none; ">
    Ho deciso di lasciare solo il termine inglese, trasformando quindi i campi in formato testuale. <br> 
    Ho eseguito questa modifica per i campi: "category", "categoryFullName", "laureates.knownName" e "laureates.motivation".
</li>
<li> Ho trasformato "awardYear" in un campo intero (originariamente testuale)</li>
<li> Ho rimosso i campi "laureates.fullName" e "laureates.nativeName" (il motivo è spiegato in seguito) </li>
</ul>

## File con i vincitori
Il file <i>JSON_Files/modificati/laureates2.json</i> contiene l'elenco dei vincitori dei premi Nobel, con la seguente struttura d'esempio:

```
// Per le persone
{
    "id": "102",
    "knownName": "Aage N. Bohr",
    "givenName": "Aage N.",
    "familyName": "Bohr",
    "fullName": "Aage Niels Bohr",
    "gender": "male",
    "birth": {
        "date": "1922-06-19",
        "place": {
            "cityNow": "Copenhagen",
            "countryNow": "Denmark"
        }
    },
    "death": {
        "date": "2009-09-08",
        "place": {
            "cityNow": "Copenhagen",
            "countryNow": "Denmark"
        }
    },
    "wikipedia": "https://en.wikipedia.org/wiki/Aage_Bohr"
} 

// Per le organizzazioni
{
    "id": "537",
    "orgName": "Amnesty International",
    "nativeName": "Amnesty International",
    "founded": {
        "date": "1961-00-00",
        "place": {
            "cityNow": "London",
            "countryNow": "United Kingdom"
        }
    },
    "wikipedia": "https://en.wikipedia.org/wiki/Amnesty_International"
}
```

### Modifiche effettuate
Rispetto al file JSON originale (contenuto in <i>JSON_Files/originali/laureates.json</i>): 
<ul>
    <li>  
        Ho rimosso i sottoalberi "/link" e "/meta" perchè contenenti informazioni non inerenti ai vincitori.
    </li>
    <li> 
        Come prima, ho rimosso la traduzione svedese e norvegese dei seguenti campi (lasciando solo quella inglese): "knownName", "givenName", "givenName", "fullName", "cityNow", "countryNow", "orgName"
    </li>
    <li>
        Ho modificato i campi "birth.place", "death.place" e "founded.place" rimuovendo i sotto-campi "city", "country", "continent" e "locationString". In questo modo ho semplificato la struttura del file, rimuovendo informazioni molto specifiche sui luoghi geografici e non inerenti al vincitore del premio.
    </li>
    <li> Ho rimosso il campo "nobelPrizes", contenente l'elenco dei premi Nobel vinti dalla persona/organizzazione, perchè già memorizzati nel file nobelPrizes2.json.
</ul>

## Struttura delle Collection
Nelle interrogazioni proposte nell'applicazione, ho dato la possibilità all'utente di scegliere se visionare i vincitori dei premi in modo sintetico (solo il nome) o dettagliato (tutti i campi). Quindi, ho deciso di non unire i file laureates2.json e nobelPrizes2.json, ma di tenerli distinti. Ho deciso, inoltre, di lasciare in nobelPrizes2.json solo i campi "laureates.knownName" e "laureates.orgName", rimuovendo quindi fullName e nativeName. <br>
<mark> DA FARE </mark>
ho inserito i seguenti indici perche ... <br>

## Applicazione
Eseguire il codice nelle celle, seguendo l'ordine proposto. <br>

### Operazioni iniziali e definizione di funzioni

Inserire nella seguente variabile il path della cartella con i file JSON:
<mark> Controllare il path con il file in locale

In [29]:
path_cartella_file_json = 'C:\\Users\\utente\\Desktop\\JSON_Nobel\\'

<mark> CAMBIA I NOMI DEI FILE DELLE COSTATI

Il seguente codice crea ed inizializza un database chiamato <i>premi_nobel</i>, formato dalle due collections <i>laureates2</i> e <i>nobelPrizes2</i> (contenenti i dati memorizzati nei relativi file JSON). <br>
<b>NB: Eseguedo questo codice, gli eventuali dati memorizzati nelle collections e non presenti nei file JSON saranno eliminati. </b> Eseguire quindi il codice nel primo avvio dell'applicazione o quando si vuole ripristinare il database.

In [30]:
import json, sys
from pymongo import MongoClient

class MiaStopExecution(Exception):
    """Il raise di questa classe provoca l'interruzione dell'esecizione della cella, senza interrompere il kernel"""
    def _render_traceback_(self):
        pass

def carica_dati(nomeFile, collection):
    ''' Funzione che carica il file json nella collection. '''
    collection.drop()
    try:
        with open(path_cartella_file_json+nomeFile, encoding='utf8') as file:
            file_data = json.load(file)
    except FileNotFoundError:
        print("Path inserito non valido! Assicurarsi di aver messo / o \\\\ alla fine del path.", file=sys.stderr)
        raise MiaStopExecution
    collection.insert_many(file_data) 

# Costanti
FILE_LAUREATES = "desktop_laureates2.json"
FILE_NOBEL_PRICES = "desktop_nobelPrizes2.json"
LOOKUP_JOIN_COLLECTIONS = { 
            '$lookup': { 
                "localField": "laureates.id",
                "from": "laureates2",
                "foreignField": "id", 
                "as": "laureates" 
            } 
        }

# Inizializzazione del database
myclient = MongoClient("mongodb://localhost:27017/")
db = myclient["premi_nobel"]
collection_laureates2 = db["laureates2"]
collection_nobelPrices2 = db["nobelPrizes2"]

carica_dati(FILE_LAUREATES, collection_laureates2) 
carica_dati(FILE_NOBEL_PRICES, collection_nobelPrices2)
print("Database inizializzato correttamente")

Database inizializzato correttamente


<mark> DA INSERIRE GLI ALTRI CAMPI SOTTO laureates (portion, sortOrder, motivation)

In [31]:
def get_info_event(dizionario):
    """Funzione che ritorna l'evento in input serializzato in una stringa"""
    ret_str = ""
    if "date" in dizionario:
        ret_str += dizionario["date"]
    if "place" in dizionario:
        ret_str += " a "+ dizionario["place"].get("cityNow", "") + ", " +dizionario["place"].get("countryNow", "")
    return ret_str


def print_vincitore(l, stampa_dettagli):
    """Funzione che stampa i dati del vincitore in input"""
    if "knownName" in l:
        print("Persona: " + str(l["knownName"]))
    if "orgName" in l:
        print("Organizzazione: " + str(l["orgName"]))

    if not stampa_dettagli:
        return
    # Dettagli persone
    if "givenName" in l or "familyName" in l or "fullName" in l:
        print("\tAltri nomi: " + str(l.get("fullName", '')) + " - " + str(l.get("givenName", '')) +" - " + str(l.get("familyName", '')))
    if "gender" in l:
        print("\tSesso: " + str(l["gender"]))
    if "birth" in l:
        print("\tNato/a il " + get_info_event(l["birth"]))
    if "death" in l:
        print("\tMorto/a il " + get_info_event(l["death"]))
    
    # Dettagli organizzazioni
    if "nativeName" in l:
        print("\tAltri nomi: " + str(l["nativeName"]))
    if "founded" in l:
        print("\tFondato/a il " + get_info_event(l["founded"]))
    
    # Dettagli comuni
    if "wikipedia" in l:
        print("\tWikipedia: " + str(l["wikipedia"]))

def print_nobels(nobel_list, stampa_dettagli):
    """Funzione che stampa il nobel dato in input (in modo dettagliato se stampa_dettagli e' true)."""
    
    def print_nobel(nobel):
        print("Anno: " + str(nobel.get('awardYear', "-")))
        print("Categoria: " + str(nobel.get('category', "-")) + " - anche detta " + str(nobel.get('categoryFullName', "")))
        print("Data: " + str(nobel.get('dateAwarded', "-")))
        print("Motivazione: " + str(nobel.get('topMotivation', "-")))
        if "prizeAmount" in nobel:
            prizeAmount = nobel['prizeAmount']
            print("Importo del premio (in corone svedesi): " + get_importo(prizeAmount))
        laureates = nobel.get('laureates', [])
        for l in laureates:
            print_vincitore(l, stampa_dettagli)
    
    for nobel in nobel_list:
        print("*************")
        print_nobel(nobel)

def chiedi_stampa_dettagli():
    return input("Stampare anche i dettagli dei vincitori (y/n)?").lower()=='y'

def get_importo(importo):
    return str(f"{importo:,.2f}")

### Interrogazioni

#### 1. Cercare tutti i vincitori dei premi Nobel di una categoria

In [33]:
print("Cercare tutti i vincitori dei premi Nobel di una categoria")

# Stampa del menu e richiesta degli input
print("Quale categoria? ")
lista_categorie = collection_nobelPrices2.distinct("category") 
i = 0
for cat in lista_categorie:
    print(str(i)+ " -- " + cat)
    i+=1
num_categoria = int(input(">> "))
stampa_dettagli = input("Stampare anche i dettagli dei vincitori (y/n)?").lower()=='y'

# Controllo dei dati in input
try:
    print("Scelta: " + lista_categorie[num_categoria] + " "+("con" if stampa_dettagli else "senza" ) + " dettagli")
except IndexError:
    print("Numero inserito non valido!")
    raise MiaStopExecution

# Interrogazione e stampa dei risultati
if stampa_dettagli:
    lista_nobel = collection_nobelPrices2.aggregate([
        {'$match': {"category":lista_categorie[num_categoria]}}, 
        LOOKUP_JOIN_COLLECTIONS
    ])
else:
    lista_nobel = collection_nobelPrices2.find({"category":lista_categorie[num_categoria]}) \
    
print_nobels(lista_nobel, stampa_dettagli)

Cercare tutti i vincitori dei premi Nobel di una categoria
Quale categoria? 
0 -- Chemistry
1 -- Economic Sciences
2 -- Literature
3 -- Peace
4 -- Physics
5 -- Physiology or Medicine
Scelta: Economic Sciences senza dettagli
*************
Anno: 1969
Categoria: Economic Sciences - anche detta The Sveriges Riksbank Prize in Economic Sciences in Memory of Alfred Nobel
Data: -
Motivazione: -
Importo del premio (in corone svedesi): 375,000.00
Persona: Ragnar Frisch
Persona: Jan Tinbergen
*************
Anno: 1970
Categoria: Economic Sciences - anche detta The Sveriges Riksbank Prize in Economic Sciences in Memory of Alfred Nobel
Data: 1970-10-26
Motivazione: -
Importo del premio (in corone svedesi): 400,000.00
Persona: Paul A. Samuelson
*************
Anno: 1971
Categoria: Economic Sciences - anche detta The Sveriges Riksbank Prize in Economic Sciences in Memory of Alfred Nobel
Data: -
Motivazione: -
Importo del premio (in corone svedesi): 450,000.00
Persona: Simon Kuznets
*************
Anno: 

#### 2. Stampare i primi Nobel in ordine (crescente o decrescente) di importo o data

In [109]:
print("Stampare i primi N Nobel degli in ordine (crescente o decrescente) di importo o di data ")

# Richiesta degli input
n_str = input("Quanti nobel stampare (premere invio per stamparli tutti)?")
try:
    n = int(n_str if n_str != "" else -1)
    if n!=-1 and n<1:
        raise ValueError
except ValueError:
    print("Errore! Numero inserito non valido")
    raise MiaStopExecution

importo_o_data=""
while(not (importo_o_data=="1" or importo_o_data=="2")):
    importo_o_data = input("Si desiderano vedere i nobel in ordine di:\n1.Importo del premio\n2.Data\n>> ")
importo_o_data = "prizeAmount" if importo_o_data == "1" else "dateAwarded"

ordine=""
while(not (ordine=="1" or ordine=="2")):
    ordine = input("Si desiderano vedere i nobel in ordine:\n1.Crescente\n2.Decrescente\n>> ")
ordine = 1 if ordine == "1" else -1

stampa_dettagli = chiedi_stampa_dettagli()

print("Hai scelto: " + 
    ("tutti i nobel" if n==-1 else "primi " + n_str + " nobel" ) 
    + " in ordine " + ("crescente" if ordine == 1 else "decrescente") + " di "
    + ("data" if importo_o_data == "dateAwarded" else "importo") + " " 
    + ("con" if stampa_dettagli else "senza" ) + " dettagli"
)

# Interrogazione e stampa dei risultati
if stampa_dettagli:
    lista_filtri = [
        { '$match': {importo_o_data:{'$exists':'true'}}},    
        { '$sort':  {importo_o_data: ordine } },
        LOOKUP_JOIN_COLLECTIONS
    ]
    if n!=-1:
        lista_filtri += [{'$limit':n}]
    lista_nobel = collection_nobelPrices2.aggregate(lista_filtri)
else:
    lista_nobel = collection_nobelPrices2.find({importo_o_data:{'$exists':'true'}}).sort(importo_o_data, ordine)
    if n != -1:
        lista_nobel = lista_nobel.limit(n)

print_nobels(lista_nobel, stampa_dettagli)

Stampare i primi N nobel degli in ordine (crescente o decrescente) di importo o di data 
Hai scelto: primi 2 nobel in ordine decrescente di importo con dettagli
*************
Anno: 2001
Categoria: Economic Sciences - anche detta The Sveriges Riksbank Prize in Economic Sciences in Memory of Alfred Nobel
Data: 2001-10-10
Motivazione: -
Importo del premio (in corone svedesi): 10,000,000.00
Persona: A. Michael Spence
	Altri nomi: A. Michael Spence - A. Michael - Spence
	Sesso: male
	Nato/a il 1943-00-00 a Montclair, NJ, USA
	Wikipedia: https://en.wikipedia.org/wiki/Michael_Spence
Persona: George A. Akerlof
	Altri nomi: George A. Akerlof - George A. - Akerlof
	Sesso: male
	Nato/a il 1940-06-17 a New Haven, CT, USA
	Wikipedia: https://en.wikipedia.org/wiki/George_Akerlof
Persona: Joseph E. Stiglitz
	Altri nomi: Joseph E. Stiglitz - Joseph E. - Stiglitz
	Sesso: male
	Nato/a il 1943-02-09 a Gary, IN, USA
	Wikipedia: https://en.wikipedia.org/wiki/Joseph_Stiglitz
*************
Anno: 2001
Categor

#### 3. Interrogazioni con max, min e avg

In [105]:
print("Interrogazioni con max, min e avg")
from math import trunc

# Interrogazione e stampa dei risultati
risultati = list(collection_nobelPrices2.aggregate([{ "$group": { 
        "_id": 'null',
        "max_data": { "$max": "$dateAwarded" } , 
        "min_data": { "$min": "$dateAwarded" } ,
        "max_prize": { "$max": "$prizeAmount" } , 
        "min_prize": { "$min": "$prizeAmount" } ,
        "avg_prize": { "$avg": "$prizeAmount"}
    }}]))[0]
print("I nobel presenti nel database:")
print("\t- sono stati vinti tra il " + risultati['min_data'] + " e il " + risultati["max_data"])
print("\t- hanno un importo compreso tra " + get_importo(risultati['min_prize']) + " e " + get_importo(risultati['max_prize']) + " (corone svedesi), per un importo medio di: " 
    + get_importo(risultati['avg_prize']))

Interrogazioni con max, min e avg
I nobel presenti nel database:
	- sono stati vinti tra il 1901-12-10 e il 2022-10-07
	- hanno un importo compreso tra 114,935.00 e 10,000,000.00 (corone svedesi), per un importo medio di: 2,866,560.87


#### 4. Quanti Nobel sono stati vinti per ogni categoria?

In [41]:
print("Quanti Nobel sono stati vinti per ogni categoria?\n")
print("Categoria\t\t\tNumero di Nobel vinti")
print("--------------------------------------------------------")

category_count = list(collection_nobelPrices2.aggregate([   
    { '$unwind': "$category" },
    { '$group': { '_id': "$category" , 'count': { '$sum': 1 } }}
]))

for elem in category_count:
    categoria = elem["_id"] 
    count = str(elem["count"])
    sep = "\t\t" if len(categoria)>16 else ("\t\t\t\t" if len(categoria)<9 else "\t\t\t")
    print(categoria +sep+ count)

Quanti Nobel sono stati vinti per ogni categoria?

Categoria			Numero di Nobel vinti
--------------------------------------------------------
Peace				122
Literature			122
Physics				122
Physiology or Medicine		122
Economic Sciences		54
Chemistry			122


#### 5. Persone/organizzazioni che hanno vinto piu di un Nobel

In [83]:
print("Persone/organizzazioni che hanno vinto piu di un Nobel")

print("Scegliere un'opzione:\n1. Considerare solo i vincitori che hanno vinto come singoli\n2. Considerare anche coloro che hanno vinto insieme ad altri vincitori")
opzione = ""
while (not(opzione=="1" or opzione=="2")):
    opzione = input(">> ")

stampa_dettagli = chiedi_stampa_dettagli()


filtri_ricerca = [
    { '$group': { '_id': "$laureates.id" , 'count': { '$sum': 1 } }},
    { '$match' : {'count': {'$gt':1}}}, 
    { '$sort' : {'count':-1}},
    { '$lookup': { 
            "localField": "_id",
            "from": "laureates2",
            "foreignField": "id", 
            "as": "vincitore" 
        } 
    }
]
if opzione == "2":
    filtri_ricerca = [{ '$unwind': "$laureates" }] + filtri_ricerca

lista_vincitori = list(collection_nobelPrices2.aggregate(filtri_ricerca))


for elem in lista_vincitori:
    if len(elem["vincitore"]) == 0:
        continue
    print("--------------")
    print("Nobel vinti: " + str(elem["count"]))
    print_vincitore(elem["vincitore"][0], stampa_dettagli)



Persone/organizzazioni che hanno vinto piu di un Nobel
Scegliere un'opzione:
1. Considerare solo i vincitori che hanno vinto come singoli
2. Considerare anche coloro che hanno vinto insieme ad altri vincitori
--------------
Nobel vinti: 2
Organizzazione: Office of the United Nations High Commissioner for Refugees
	Altri nomi: Office of the United Nations High Commissioner for Refugees
	Fondato/a il 1950-12-14 a Geneva, Switzerland
	Wikipedia: https://en.wikipedia.org/wiki/United_Nations_High_Commissioner_for_Refugees
--------------
Nobel vinti: 2
Organizzazione: International Committee of the Red Cross
	Altri nomi: Comité international de la Croix Rouge
	Fondato/a il 1863-00-00 a Geneva, Switzerland
	Wikipedia: https://en.wikipedia.org/wiki/International_Committee_of_the_Red_Cross
--------------
Nobel vinti: 2
Persona: Linus Pauling
	Altri nomi: Linus Carl Pauling - Linus - Pauling
	Sesso: male
	Nato/a il 1901-02-28 a Portland, OR, USA
	Morto/a il 1994-08-19 a Big Sur, CA, USA
	Wikiped

#### 6. Sesta interrogazione
Con comandi non visti a lezione

### Scritture e modifiche

#### 1. Inserire un premio nobel ed eventualmente dati del laureates

#### 2. Modifica/aggiunta di campi di un premio nobel